#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager
import bootcampviztools as bt
import statsmodels.api as sm
import warnings
import sklearn
import os
import cv2
import re

from matplotlib import style
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel, RFE, SequentialFeatureSelector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet
from sklearn.metrics import classification_report, ConfusionMatrixDisplay,  mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, silhouette_score, r2_score, silhouette_samples, mutual_info_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder, scale, FunctionTransformer, OneHotEncoder
from matplotlib.ticker import FixedLocator, FixedFormatter
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.utils import resample
from scipy import stats
from scipy.stats import mannwhitneyu

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor


warnings.filterwarnings('ignore')
style.use('ggplot') or plt.style.use('ggplot')

## Data

In [7]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
pcs = pd.read_csv("./data/train.csv")

In [8]:
target = "Price_euros"

In [9]:
var_desc = ["laptop_ID", "Product"] 
pcs = pcs.drop(["laptop_ID", "Product"], axis=1) 

X = pcs.drop("Price_euros", axis=1) 
y = pcs["Price_euros"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
X_train['Inches'] = X_train['Inches'].astype(float)

In [11]:
X_train['Weight'] = X_train['Weight'].apply(lambda x: float(x.replace('kg', '').replace(',', '.')))

In [12]:
X_train['Ram'] = X_train['Ram'].apply(lambda x: int(x.replace('GB', '')))

In [13]:
marcas_ppales = X_train['Company'].value_counts().sort_values(ascending=False) / X_train['Company'].count()
top_10_companies = marcas_ppales[marcas_ppales >= 0.01].index.tolist()
otras_marcas = [company for company in pcs['Company'].unique() if company not in top_10_companies]
X_train['Company'] = X_train['Company'].replace(otras_marcas, "Otra")

In [14]:
def extract_screen_resolution(df):
    def extract_resolution(screen_resolution):
        # Definir el patrón para buscar números de 3 a 4 dígitos separados por 'x'
        pattern = r'\b(\d{3,4})\s*x\s*(\d{3,4})\b'
        match = re.search(pattern, screen_resolution)
        if match:
            width = int(match.group(1))
            height = int(match.group(2))
            # Calcular el producto de anchura y altura
            resolution = width * height
            return resolution
        else:
            return 1

    # Definir las palabras clave para buscar en la descripción de la pantalla
    labels = ['Touchscreen', 'FullHD', 'Retina', 'HD+', '4K', 'IPS Panel']

    # Función para crear las columnas de características específicas de la pantalla
    def extract_screen_features(screen_resolution, labels):
        features = [1 if label in screen_resolution else 0 for label in labels]
        return features

    # Aplicar la función de extracción de resolución de pantalla
    df['Resolution'] = df['ScreenResolution'].apply(extract_resolution)

    # Aplicar la función de extracción de características específicas de la pantalla
    df[labels] = pd.DataFrame(df['ScreenResolution'].apply(lambda x: extract_screen_features(x, labels)).tolist(), index=df.index)

    return df

# Aplicar la función a tu DataFrame
X_train = extract_screen_resolution(X_train)

In [15]:
# Transformar la columna 'Cpu'
def extract_cpu_info(cpu):
    brand_pattern = r'(Intel|AMD)'
    speed_pattern = r'(\d+\.\d+)GHz'
    model_pattern = r'\b(?:Core [iI]\d+|Ryzen \d+)\b'
    series_pattern = r'(?:Intel|AMD)(.*?)(?:\b(?:Core [iI]\d+|Ryzen \d+|$)|\s\d+\.\d+GHz)'

    try:
        brand_match = re.search(brand_pattern, cpu)
        brand = brand_match.group(1) if brand_match else "Desconocida"

        speed_match = re.search(speed_pattern, cpu)
        speed = float(speed_match.group(1)) if speed_match else 1

        model_match = re.search(model_pattern, cpu)
        model = model_match.group() if model_match else "Desconocido"

        series_match = re.search(series_pattern, cpu)
        series = series_match.group(1).strip() if series_match else "Desconocida"

        return brand, speed, model, series
    except Exception as e:
        print("Error durante la extracción de información de la CPU:", e)
        return "Desconocida", -1.0, "Desconocido", "Desconocida"

# Aplicar la función extract_cpu_info a cada fila del DataFrame
for index, row in X_train.iterrows():
    cpu_info = extract_cpu_info(row['Cpu'])
    
    # Asignar los valores extraídos a las nuevas columnas en la misma fila
    X_train.at[index, 'CPU_marca'] = cpu_info[0]
    X_train.at[index, 'CPU_velocidad'] = cpu_info[1]
    X_train.at[index, 'CPU_modelo'] = cpu_info[2]
    X_train.at[index, 'CPU_serie'] = cpu_info[3]

In [16]:
# Transformar la columna 'Memory'
def extract_memory_info(memory):
    memory_pattern = r'(\d+)\s*(?:GB|T|TB)'
    keyword_pattern = r'HDD|SSD|Flash Storage|Hybrid'

    match = re.findall(memory_pattern, memory)
    if match:
        memory_value = int(match[0][0]) * 1024 if 'T' in memory else int(match[0][0])
    else:
        memory_value = 0

    keywords = re.findall(keyword_pattern, memory)
    storage_dict = {'HDD': 0, 'SSD': 0, 'Flash Storage': 0, 'Hybrid': 0}
    for keyword in keywords:
        storage_dict[keyword] = memory_value

    return storage_dict['HDD'], storage_dict['SSD'], storage_dict['Flash Storage'], storage_dict['Hybrid']

X_train['Memo_HDD'], X_train['Memo_SSD'], X_train['Memo_Flash'], X_train['Memo_Hybrid'] = zip(*X_train['Memory'].apply(extract_memory_info))
X_train.rename(columns={'Memo_HDD': 'Memo_HDD_GB', 'Memo_SSD': 'Memo_SSD_GB', 'Memo_Flash': 'Memo_Flash_GB', 'Memo_Hybrid': 'Memo_Hybrid_GB'}, inplace=True)


In [18]:
var_fuera = []
cardinality = X_train.select_dtypes(include='object').nunique()

# Definir un umbral para la cardinalidad (por ejemplo, 10)
umbral_cardinalidad = 15

# Identificar las columnas categóricas con una cardinalidad alta
columnas_alta_cardinalidad = cardinality[cardinality > umbral_cardinalidad].index.tolist()

# Mover las columnas con alta cardinalidad a var_fuera y eliminarlas del DataFrame principal
var_fuera.append(columnas_alta_cardinalidad)
X_train.drop(columns=columnas_alta_cardinalidad, inplace=True)

In [19]:
columnas_categoricas = X_train.select_dtypes(include='object').columns

# Seleccionar solo las columnas numéricas que quedan en pcs_eda
columnas_numericas = X_train.select_dtypes(include=['int', 'float']).columns

# Aplicar get_dummies a las columnas categóricas seleccionadas
X_train_encoded = pd.get_dummies(X_train, columns=columnas_categoricas, drop_first=True)

In [20]:
# Crear un objeto StandardScaler
scaler = StandardScaler()

# Escalar las columnas numéricas seleccionadas en X_train_encoded
X_train_encoded[columnas_numericas] = scaler.fit_transform(X_train_encoded[columnas_numericas])

Ahora automatizar esas transformaciones

In [41]:
class DataFrameTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.top_10_companies = None
        self.scaler = None

    def fit(self, X, y=None):
        # Determinar las top 10 companies
        print("Columnas en fit:", X.columns)
        marcas_ppales = X['Company'].value_counts().sort_values(ascending=False) / X['Company'].count()
        self.top_10_companies = marcas_ppales[marcas_ppales >= 0.01].index.tolist()
        return self

    def transform(self, X, y=None):
        # Verificar si la columna 'Company' está presente
        print("Columnas antes de transform:", X.columns)
        if 'Company' not in X.columns:
            raise ValueError("La columna 'Company' no está presente en el dataset.")
        
        # Eliminar columnas innecesarias
        if 'laptop_ID' in X.columns:
            X = X.drop(columns=['laptop_ID'])
        if 'Product' in X.columns:
            X = X.drop(columns=['Product'])
        print("Columnas después de eliminar columnas innecesarias:", X.columns)
        
        # Convertir tipos de datos
        if 'Inches' in X.columns:
            X['Inches'] = X['Inches'].astype(float)
        if 'Weight' in X.columns:
            X['Weight'] = X['Weight'].apply(lambda x: float(x.replace('kg', '').replace(',', '.')))
        if 'Ram' in X.columns:
            X['Ram'] = X['Ram'].apply(lambda x: int(x.replace('GB', '')))
        
        # Reemplazar valores en Company
        otras_marcas = [company for company in X['Company'].unique() if company not in self.top_10_companies]
        X['Company'] = X['Company'].replace(otras_marcas, "Otra")
        print("Columnas después de transformar 'Company':", X.columns)
        
        # Extraer características de la pantalla
        X = self.extract_screen_resolution(X)
        print("Columnas después de extraer características de la pantalla:", X.columns)
        
        # Extraer información del CPU
        X = self.extract_cpu_info(X)
        print("Columnas después de extraer información del CPU:", X.columns)
        
        # Extraer información de la memoria
        X = self.extract_memory_info(X)
        print("Columnas después de extraer información de la memoria:", X.columns)
        
        # Eliminar outliers
        X = self.remove_outliers(X)
        print("Columnas después de eliminar outliers:", X.columns)
        
        # Eliminar columnas con alta cardinalidad
        X = self.remove_high_cardinality(X)
        print("Columnas después de eliminar columnas con alta cardinalidad:", X.columns)
        
        # Codificar variables categóricas y escalar variables numéricas
        X = pd.get_dummies(X, drop_first=True)
        print("Columnas después de codificar variables categóricas:", X.columns)
        if self.scaler is None:
            self.scaler = StandardScaler()
            X[X.select_dtypes(include=[np.number]).columns] = self.scaler.fit_transform(X.select_dtypes(include=[np.number]))
        else:
            X[X.select_dtypes(include=[np.number]).columns] = self.scaler.transform(X.select_dtypes(include=[np.number]))
        print("Columnas después de escalar variables numéricas:", X.columns)
        
        return X

    def extract_screen_resolution(self, df):
        def extract_resolution(screen_resolution):
            pattern = r'\b(\d{3,4})\s*x\s*(\d{3,4})\b'
            match = re.search(pattern, screen_resolution)
            if match:
                width = int(match.group(1))
                height = int(match.group(2))
                resolution = width * height
                return resolution
            else:
                return 1

        labels = ['Touchscreen', 'FullHD', 'Retina', 'HD+', '4K', 'IPS Panel']
        
        def extract_screen_features(screen_resolution, labels):
            features = [1 if label in screen_resolution else 0 for label in labels]
            return features

        df['Resolution'] = df['ScreenResolution'].apply(extract_resolution)
        df[labels] = pd.DataFrame(df['ScreenResolution'].apply(lambda x: extract_screen_features(x, labels)).tolist(), index=df.index)
        
        return df

    def extract_cpu_info(self, df):
        def extract_cpu_info(cpu):
            brand_pattern = r'(Intel|AMD)'
            speed_pattern = r'(\d+\.\d+)GHz'
            model_pattern = r'\b(?:Core [iI]\d+|Ryzen \d+)\b'
            series_pattern = r'(?:Intel|AMD)(.*?)(?:\b(?:Core [iI]\d+|Ryzen \d+|$)|\s\d+\.\d+GHz)'

            try:
                brand_match = re.search(brand_pattern, cpu)
                brand = brand_match.group(1) if brand_match else "Desconocida"

                speed_match = re.search(speed_pattern, cpu)
                speed = float(speed_match.group(1)) if speed_match else 1

                model_match = re.search(model_pattern, cpu)
                model = model_match.group() if model_match else "Desconocido"

                series_match = re.search(series_pattern, cpu)
                series = series_match.group(1).strip() if series_match else "Desconocida"

                return brand, speed, model, series
            except Exception as e:
                print("Error durante la extracción de información de la CPU:", e)
                return "Desconocida", -1.0, "Desconocido", "Desconocida"

        for index, row in df.iterrows():
            cpu_info = extract_cpu_info(row['Cpu'])
            df.at[index, 'CPU_marca'] = cpu_info[0]
            df.at[index, 'CPU_velocidad'] = cpu_info[1]
            df.at[index, 'CPU_modelo'] = cpu_info[2]
            df.at[index, 'CPU_serie'] = cpu_info[3]
        
        return df

    def extract_memory_info(self, df):
        def extract_memory_info(memory):
            memory_pattern = r'(\d+)\s*(?:GB|T|TB)'
            keyword_pattern = r'HDD|SSD|Flash Storage|Hybrid'

            match = re.findall(memory_pattern, memory)
            if match:
                memory_value = int(match[0][0]) * 1024 if 'T' in memory else int(match[0][0])
            else:
                memory_value = 0

            keywords = re.findall(keyword_pattern, memory)
            storage_dict = {'HDD': 0, 'SSD': 0, 'Flash Storage': 0, 'Hybrid': 0}
            for keyword in keywords:
                storage_dict[keyword] = memory_value

            return storage_dict['HDD'], storage_dict['SSD'], storage_dict['Flash Storage'], storage_dict['Hybrid']

        df['Memo_HDD'], df['Memo_SSD'], df['Memo_Flash'], df['Memo_Hybrid'] = zip(*df['Memory'].apply(extract_memory_info))
        df.rename(columns={'Memo_HDD': 'Memo_HDD_GB', 'Memo_SSD': 'Memo_SSD_GB', 'Memo_Flash': 'Memo_Flash_GB', 'Memo_Hybrid': 'Memo_Hybrid_GB'}, inplace=True)
        
        return df    

    def remove_high_cardinality(self, df, threshold=15):
        cardinality = df.select_dtypes(include='object').nunique()
        high_card_cols = cardinality[cardinality > threshold].index.tolist()
        df.drop(columns=high_card_cols, inplace=True)
        
        return df

### Modelos

In [22]:
model_names = ["Regresion Lineal","DecisionTree","Random Forest","XGBoost","LightGBM","CatBoost"]
lr_reg = LinearRegression()
tree_reg = DecisionTreeRegressor(random_state= 42)
rf_reg = RandomForestRegressor(random_state= 42)
xgb_reg = XGBRegressor(random_state = 42)
lgb_reg = LGBMRegressor(random_state= 42, verbose = -100)
cat_reg = CatBoostRegressor(random_state= 42, verbose = False)

model_set = [lr_reg, tree_reg, rf_reg, xgb_reg, lgb_reg, cat_reg]

In [23]:
metricas_cv = {}

# Lista para almacenar los valores medios de las métricas
valores = []

# Iterar sobre cada modelo en el conjunto de modelos
for nombre, modelo in zip(model_names, model_set):
    print(modelo)
    # Calcular las métricas de validación cruzada para el modelo actual
    metricas_cv[nombre] = cross_val_score(modelo, X_train_encoded, y_train, cv=5, scoring="neg_mean_squared_error")
    
    # Calcular el valor medio de las métricas y añadirlo a la lista de valores
    valores.append(np.mean(metricas_cv[nombre]))

# Identificar el modelo ganador basado en el valor medio de las métricas
ganador = list(metricas_cv.keys())[np.argmax(valores)]

LinearRegression()
DecisionTreeRegressor(random_state=42)
RandomForestRegressor(random_state=42)
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
LGBMRegressor(random_state=42, verbose=-100)


In [24]:
for model_name, valores in metricas_cv.items():
    print(f"Modelo <{model_name}>, MAE_CV: {-np.mean(valores)}")
print(f"El ganador es {ganador}")

Modelo <Regresion Lineal>, MAE_CV: 69067948.67608929
Modelo <DecisionTree>, MAE_CV: 205921.39731346339
Modelo <Random Forest>, MAE_CV: 100146.55816968155
Modelo <XGBoost>, MAE_CV: 101366.35936131825
Modelo <LightGBM>, MAE_CV: 117698.80911356094
Modelo <CatBoost>, MAE_CV: 86843.41017753242
El ganador es CatBoost


In [25]:
# Ajustes CatBoost

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'iterations': [100, 200, 300]
}

# Inicializa un regresor CatBoost
catboost_reg = CatBoostRegressor()

# Configura el GridSearchCV con el regresor CatBoost, los hiperparámetros y la métrica de evaluación
grid_search = GridSearchCV(catboost_reg,
                           param_grid,
                           cv=5,  # Número de pliegues para la validación cruzada
                           scoring='neg_mean_absolute_error',  # Métrica de evaluación
                           n_jobs=-1
                          )

# Ajusta el GridSearchCV para encontrar el mejor modelo
grid_search.fit(X_train_encoded, y_train)

0:	learn: 689.6525090	total: 3.05ms	remaining: 911ms
1:	learn: 659.4164960	total: 5.21ms	remaining: 776ms
2:	learn: 624.1698949	total: 7.25ms	remaining: 718ms
3:	learn: 593.6912843	total: 9.04ms	remaining: 669ms
4:	learn: 566.9646254	total: 10.9ms	remaining: 643ms
5:	learn: 542.4606939	total: 12.9ms	remaining: 635ms
6:	learn: 518.9665903	total: 15.4ms	remaining: 645ms
7:	learn: 498.9405938	total: 17.7ms	remaining: 646ms
8:	learn: 480.7192610	total: 19.7ms	remaining: 637ms
9:	learn: 464.8284319	total: 21ms	remaining: 609ms
10:	learn: 449.3423447	total: 22.8ms	remaining: 600ms
11:	learn: 434.4616329	total: 24.8ms	remaining: 595ms
12:	learn: 422.6427977	total: 26.9ms	remaining: 593ms
13:	learn: 409.3647590	total: 28.7ms	remaining: 587ms
14:	learn: 397.8599015	total: 30.6ms	remaining: 581ms
15:	learn: 387.6245739	total: 32.4ms	remaining: 576ms
16:	learn: 377.6571218	total: 34.3ms	remaining: 572ms
17:	learn: 368.1882990	total: 36.1ms	remaining: 565ms
18:	learn: 361.4657005	total: 38.4ms	rem

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x000002B4BD5A32B0>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 200, 300],
                         'learning_rate': [0.01, 0.05, 0.1]},
             scoring='neg_mean_absolute_error')

In [26]:
# Obtiene el mejor modelo CatBoost encontrado
mejor_modelo_catboost = grid_search.best_estimator_

# Realiza predicciones en el conjunto de entrenamiento
predicciones_catboost_train = mejor_modelo_catboost.predict(X_train_encoded)

# Calcula el MAE en el conjunto de entrenamiento
mae_catboost_train = mean_absolute_error(y_train, predicciones_catboost_train)
print("MAE en el conjunto de entrenamiento usando CatBoost:", mae_catboost_train)

MAE en el conjunto de entrenamiento usando CatBoost: 81.1271281928112


In [27]:
# Obtiene el mejor modelo encontrado
mejor_modelo_catboost = grid_search.best_estimator_

# Realiza predicciones en el conjunto de entrenamiento
predicciones_catboost_train = mejor_modelo_catboost.predict(X_train_encoded)

# Calcula el MAE en el conjunto de entrenamiento
mae_catboost_train = mean_absolute_error(y_train, predicciones_catboost_train)
print("MAE en el conjunto de entrenamiento usando CatBoost:", mae_catboost_train)

MAE en el conjunto de entrenamiento usando CatBoost: 81.1271281928112


Vamos a mejorar los otros modelos

In [28]:
# Ajustes DecisionTreeRegressor

param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inicializa un regresor DecisionTree
decision_tree_reg = DecisionTreeRegressor()

# Configura el GridSearchCV con el regresor DecisionTree, los hiperparámetros y la métrica de evaluación
grid_search = GridSearchCV(decision_tree_reg,
                           param_grid,
                           cv=5,  # Número de pliegues para la validación cruzada
                           scoring='neg_mean_absolute_error',  # Métrica de evaluación
                           n_jobs=-1
                          )

# Ajusta el GridSearchCV para encontrar el mejor modelo
grid_search.fit(X_train_encoded, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_absolute_error')

In [29]:
# Obtiene el mejor modelo DecisionTree encontrado
mejor_modelo_decisiontree = grid_search.best_estimator_

# Realiza predicciones en el conjunto de entrenamiento
predicciones_decisiontree_train = mejor_modelo_decisiontree.predict(X_train_encoded)

# Calcula el MAE en el conjunto de entrenamiento
mae_decisiontree_train = mean_absolute_error(y_train, predicciones_decisiontree_train)
print("MAE en el conjunto de entrenamiento usando CatBoost:", mae_decisiontree_train)

MAE en el conjunto de entrenamiento usando CatBoost: 218.4317141189976


In [30]:
# Ajustes RandomForest

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inicializa un regresor RandomForest
random_forest_reg = RandomForestRegressor()

# Configura el GridSearchCV con el regresor RandomForest, los hiperparámetros y la métrica de evaluación
grid_search = GridSearchCV(random_forest_reg,
                           param_grid,
                           cv=5,  # Número de pliegues para la validación cruzada
                           scoring='neg_mean_absolute_error',  # Métrica de evaluación
                           n_jobs=-1
                          )

# Ajusta el GridSearchCV para encontrar el mejor modelo
grid_search.fit(X_train_encoded, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_absolute_error')

In [31]:
# Obtiene el mejor modelo RandomForest encontrado
mejor_modelo_RandomForest = grid_search.best_estimator_

# Realiza predicciones en el conjunto de entrenamiento
predicciones_RandomForest_train = mejor_modelo_RandomForest.predict(X_train_encoded)

# Calcula el MAE en el conjunto de entrenamiento
mae_RandomForest_train = mean_absolute_error(y_train, predicciones_RandomForest_train)
print("MAE en el conjunto de entrenamiento usando RandomForest:", mae_RandomForest_train)

MAE en el conjunto de entrenamiento usando RandomForest: 73.95137654362742


In [32]:
# Ajustes XGBoost

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7]
}

# Inicializa un regresor XGBoost
xgboost_reg = XGBRegressor()

# Configura el GridSearchCV con el regresor XGBoost, los hiperparámetros y la métrica de evaluación
grid_search = GridSearchCV(xgboost_reg,
                           param_grid,
                           cv=5,  # Número de pliegues para la validación cruzada
                           scoring='neg_mean_absolute_error',  # Métrica de evaluación
                           n_jobs=-1
                          )

# Ajusta el GridSearchCV para encontrar el mejor modelo
grid_search.fit(X_train_encoded, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_absolute_error')

In [33]:
# Obtiene el mejor modelo XGBoost encontrado
mejor_modelo_XGBoost = grid_search.best_estimator_

# Realiza predicciones en el conjunto de entrenamiento
predicciones_XGBoost_train = mejor_modelo_XGBoost.predict(X_train_encoded)

# Calcula el MAE en el conjunto de entrenamiento
mae_XGBoost_train = mean_absolute_error(y_train, predicciones_XGBoost_train)
print("MAE en el conjunto de entrenamiento usando XGBoost:", mae_XGBoost_train)

MAE en el conjunto de entrenamiento usando XGBoost: 47.09261759691278


In [34]:
# Ajustes LightGBM

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'num_leaves': [15, 31, 63]  # Prueba diferentes valores para num_leaves
}

# Inicializa un regresor LightGBM
lgbm_reg = LGBMRegressor()

# Configura el GridSearchCV con el regresor LightGBM, los hiperparámetros y la métrica de evaluación
grid_search = GridSearchCV(lgbm_reg,
                           param_grid,
                           cv=5,  # Número de pliegues para la validación cruzada
                           scoring='neg_mean_absolute_error',  # Métrica de evaluación
                           n_jobs=-1
                          )

# Ajusta el GridSearchCV para encontrar el mejor modelo
grid_search.fit(X_train_encoded, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000387 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 457
[LightGBM] [Info] Number of data points in the train set: 729, number of used features: 30
[LightGBM] [Info] Start training from score 1122.043223
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

GridSearchCV(cv=5, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300],
                         'num_leaves': [15, 31, 63]},
             scoring='neg_mean_absolute_error')

In [35]:
# Obtiene el mejor modelo LightGBM encontrado
mejor_modelo_LightGBM = grid_search.best_estimator_

# Realiza predicciones en el conjunto de entrenamiento
predicciones_LightGBM_train = mejor_modelo_LightGBM.predict(X_train_encoded)

# Calcula el MAE en el conjunto de entrenamiento
mae_LightGBM_train = mean_absolute_error(y_train, predicciones_LightGBM_train)
print("MAE en el conjunto de entrenamiento usando LightGBM:", mae_LightGBM_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
MAE en el conjunto de entrenamiento usando LightGBM: 87.734745638108


## Test

Ahora modelos contra test

In [42]:
# Ejemplo de uso:
transformer = DataFrameTransformer()

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('transformer', transformer)
    # Aquí puedes agregar tu modelo
    # ('model', RandomForestRegressor())
])

# Ajustar el pipeline
pipeline.fit(X_train_encoded, y_train)

# Transformar el dataset de prueba
X_test_transformed = pipeline.transform(X_test)

Columnas en fit: Index(['id', 'Inches', 'Ram', 'Weight', 'Resolution', 'Touchscreen', 'FullHD',
       'Retina', 'HD+', '4K', 'IPS Panel', 'CPU_velocidad', 'Memo_HDD_GB',
       'Memo_SSD_GB', 'Memo_Flash_GB', 'Memo_Hybrid_GB', 'Company_Apple',
       'Company_Asus', 'Company_Dell', 'Company_HP', 'Company_Lenovo',
       'Company_MSI', 'Company_Otra', 'Company_Toshiba', 'TypeName_Gaming',
       'TypeName_Netbook', 'TypeName_Notebook', 'TypeName_Ultrabook',
       'TypeName_Workstation', 'OpSys_Chrome OS', 'OpSys_Linux',
       'OpSys_Mac OS X', 'OpSys_No OS', 'OpSys_Windows 10',
       'OpSys_Windows 10 S', 'OpSys_Windows 7', 'OpSys_macOS',
       'CPU_marca_Intel', 'CPU_modelo_Core i5', 'CPU_modelo_Core i7',
       'CPU_modelo_Desconocido', 'CPU_modelo_Ryzen 1600',
       'CPU_modelo_Ryzen 1700'],
      dtype='object')


KeyError: 'Company'

In [ ]:
# Realiza predicciones en el conjunto de entrenamiento
predicciones_catboost_train = mejor_modelo_catboost.predict(X_test_transformed)

# Calcula el MAE en el conjunto de entrenamiento
mae_catboost_train = mean_absolute_error(y_test_no_outliers, predicciones_catboost_train)
print("MAE en el conjunto de entrenamiento usando CatBoost:", mae_catboost_train)

NameError: name 'X_test_transformed' is not defined

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

### 2. Dividir X_train, X_test, y_train, y_test

### 3. Crear y entrenar el modelo


### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

In [ ]:
X = pcs.drop(columns=['Price_euros'])  # Ajusta 'target_column' al nombre de tu variable objetivo
y = pcs['Price_euros']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Ajusta el tamaño del conjunto de prueba según tus necesidades

X_train_transformed = transformer_pipeline.fit_transform(X_train)
X_test_transformed = transformer_pipeline.transform(X_test)

mejor_modelo_catboost.fit(X_train_transformed, y_train)

predicciones_catboost_test = mejor_modelo_catboost.predict(X_test_transformed)

mae_catboost_test = mean_absolute_error(y_test, predicciones_catboost_test)
rmse_catboost_test = np.sqrt(mean_squared_error(y_test, predicciones_catboost_test))
r2_catboost_test = r2_score(y_test, predicciones_catboost_test)

# Imprime las métricas de evaluación
print("MAE en el conjunto de prueba usando CatBoost:", mae_catboost_test)
print("RMSE en el conjunto de prueba usando CatBoost:", rmse_catboost_test)
print("R^2 en el conjunto de prueba usando CatBoost:", r2_catboost_test)


Valor de CPU (antes de procesamiento): Intel Core i3 6006U 2GHz
Valor de CPU (antes de procesamiento): AMD A9-Series 9420 3GHz
Valor de CPU (antes de procesamiento): Intel Pentium Quad Core N3710 1.6GHz
Valor de CPU (antes de procesamiento): Intel Core i5 8250U 1.6GHz
Valor de CPU (antes de procesamiento): Intel Pentium Quad Core N3710 1.6GHz
Valor de CPU (antes de procesamiento): Intel Core i5 7200U 2.5GHz
Valor de CPU (antes de procesamiento): Intel Core i7 6600U 2.6GHz
Valor de CPU (antes de procesamiento): Intel Core M 1.2GHz
Valor de CPU (antes de procesamiento): Intel Core i3 7100U 2.4GHz
Valor de CPU (antes de procesamiento): Intel Core M 6Y75 1.2GHz
Valor de CPU (antes de procesamiento): Intel Core i7 7820HK 2.9GHz
Valor de CPU (antes de procesamiento): Intel Core i5 7200U 2.5GHz
Valor de CPU (antes de procesamiento): Intel Core i7 6600U 2.6GHz
Valor de CPU (antes de procesamiento): Intel Core i5 7200U 2.5GHz
Valor de CPU (antes de procesamiento): Intel Core i5 6200U 2.3GHz
Val

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Company_Huawei
- Cpu_AMD E-Series 9000 2.2GHz
- Cpu_AMD E-Series 9000e 1.5GHz
- Cpu_AMD E-Series E2-6110 1.5GHz
- Cpu_Intel Atom X5-Z8350 1.44GHz
- ...
Feature names seen at fit time, yet now missing:
- Company_Chuwi
- Company_LG
- Company_Microsoft
- Cpu_AMD A10-Series A10-9620P 2.5GHz
- Cpu_AMD A12-Series 9700P 2.5GHz
- ...


### 2. Carga los datos de `test.csv` para predecir.

In [ ]:
test = pd.read_csv("./data/test.csv")

In [ ]:
test_final = preprocess_data(test)

NameError: name 'preprocess_data' is not defined

In [ ]:
# Leer el archivo CSV
try:
    test = pd.read_csv("./data/test.csv")
    print("Archivo CSV cargado exitosamente.")
except FileNotFoundError:
    print("Error: El archivo 'test.csv' no se encuentra en la ruta especificada.")
except pd.errors.EmptyDataError:
    print("Error: El archivo 'test.csv' está vacío.")
except pd.errors.ParserError:
    print("Error: Hubo un problema al analizar el archivo 'test.csv'.")
except Exception as e:
    print(f"Error inesperado: {e}")

# Asegúrate de que la función preprocess_data esté definida antes de este punto
try:
    test_final = preprocess_data(test)
    print("Preprocesamiento completado exitosamente.")
except NameError:
    print("Error: La función 'preprocess_data' no está definida.")
except Exception as e:
    print(f"Error inesperado durante el preprocesamiento: {e}")

Archivo CSV cargado exitosamente.
Error inesperado durante el preprocesamiento: preprocess_data() missing 2 required positional arguments: 'cat_cols' and 'num_cols'


**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [ ]:
predictions = model.predict(X_pred)
predictions

NameError: name 'model' is not defined

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
sample = pd.read_csv("Dataset/sample_submission.csv") # Esta es mi ruta del archivo, la vuestra puede ser otra

In [ ]:
sample.head()

In [ ]:
sample.shape

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [ ]:
chequeator(submission) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

You're ready to submit!
